In [ ]:
!pip install xmltodict

In [ ]:
import xml.etree.ElementTree as ET

# Parse the OSM file
osm_file = '/content/planet_-8.044,31.623_-7.995,31.655.osm'
tree = ET.parse(osm_file)
root = tree.getroot()

# Extract nodes
nodes = {}
for node in root.findall('node'):
    node_id = node.get('id')
    lat = float(node.get('lat'))
    lon = float(node.get('lon'))
    nodes[node_id] = (lat, lon)

# Extract ways (routes)
routes = []
for way in root.findall('way'):
    route = []
    for nd in way.findall('nd'):
        ref = nd.get('ref')
        if ref in nodes:
            route.append(nodes[ref])  # Add latitude and longitude
    if route:
        routes.append(route)

print(f"Extracted {len(routes)} routes and {len(nodes)} nodes.")


Extracted 12210 routes and 67696 nodes.


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Flatten all coordinates to find min/max
all_coords = np.array([coord for route in routes for coord in route])
min_lat, max_lat = all_coords[:, 0].min(), all_coords[:, 0].max()
min_lon, max_lon = all_coords[:, 1].min(), all_coords[:, 1].max()

# Normalize each route
normalized_routes = [
    [( (lat - min_lat) / (max_lat - min_lat), (lon - min_lon) / (max_lon - min_lon) ) for lat, lon in route]
    for route in routes
]

# Pad the sequences
max_route_length = max(len(route) for route in normalized_routes)
padded_routes = pad_sequences(normalized_routes, maxlen=max_route_length, dtype='float32', padding='post', value=(0, 0))

# Compute mean and std excluding (0, 0)
valid_data = padded_routes[padded_routes != 0]
mean = np.mean(valid_data, axis=0)
std = np.std(valid_data, axis=0)

# Normalize
padded_routes = (padded_routes - mean) / std



In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam

input_dim = 2  # Latitude and longitude
timesteps = max_route_length

# Encoder
inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(64, activation='relu')(inputs)

# Bottleneck
bottleneck = RepeatVector(timesteps)(encoded)

# Decoder
decoded = LSTM(64, activation='relu', return_sequences=True)(bottleneck)
outputs = TimeDistributed(Dense(input_dim))(decoded)

# Model
autoencoder = Model(inputs, outputs)
autoencoder.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')
autoencoder.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 200, 2)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 64)                  │          17,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector_2 (RepeatVector)       │ (None, 200, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 200, 64)             │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 200, 2)              │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,306 (196.51 KB)

 Trainable params: 50,306 (196.51 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
autoencoder.fit(padded_routes, padded_routes, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 124s 367ms/step - loss: 1.1571 - val_loss: 0.0702
Epoch 2/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 135s 345ms/step - loss: 0.0694 - val_loss: 0.0695
Epoch 3/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 102s 334ms/step - loss: 0.0707 - val_loss: 0.0688
Epoch 4/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 144s 340ms/step - loss: 0.0697 - val_loss: 0.0682
Epoch 5/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 140s 336ms/step - loss: 0.0678 - val_loss: 0.0675
Epoch 6/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 142s 337ms/step - loss: 0.0700 - val_loss: 0.0667
Epoch 7/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 105s 342ms/step - loss: 0.0673 - val_loss: 0.0664
Epoch 8/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 139s 333ms/step - loss: 0.0689 - val_loss: 0.0652
Epoch 9/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 145s 341ms/step - loss: 0.0680 - val_loss: 0.0644
Epoch 10/10
306/306 ━━━━━━━━━━━━━━━━━━━━ 105s 342ms/step - loss: 0.0680 - val_loss: 0.0643


In [ ]:
new_route = [(31.6324805, -7.9909368), (31.6324911, -7.9909842), (31.6324564, -7.9909938)]

# Normalize
normalized_new_route = [
    ((lat - min_lat) / (max_lat - min_lat), (lon - min_lon) / (max_lon - min_lon))
    for lat, lon in new_route
]

# Pad
padded_new_route = pad_sequences([normalized_new_route], maxlen=max_route_length, dtype='float32', padding='post', value=(0, 0))

# Predict and classify
reconstructed_new_route = autoencoder.predict(padded_new_route)
new_error = np.mean(np.abs(padded_new_route - reconstructed_new_route))
print("Is Anomaly:", new_error > threshold)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Is Anomaly: False


In [ ]:
import joblib

# For TensorFlow/Keras
autoencoder.save('model.h5')

In [ ]:
import tensorflow as tf

# Specify the custom objects used in the model
custom_objects = {"mse": tf.keras.losses.MeanSquaredError()}

# Load the model with custom objects
model = tf.keras.models.load_model('/content/model.h5', custom_objects=custom_objects)

# Save it as a SavedModel
tf.saved_model.save(model, 'saved_model/')

In [ ]:
!zip -r /content/saved_model.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/assets/ (stored 0%)
  adding: content/saved_model/variables/ (stored 0%)
  adding: content/saved_model/variables/variables.index (deflated 52%)
  adding: content/saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/saved_model/fingerprint.pb (stored 0%)
  adding: content/saved_model/saved_model.pb (deflated 94%)


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.1
